# `pyhf` Demo

# Hello World, `pyhf` style

Two bin counting experiment with a background uncertainty:

In [1]:
import pyhf
import pyhf.simplemodels
import pyhf.utils

In [2]:
pdf = pyhf.simplemodels.hepdata_like(signal_data=[12.,11.], bkg_data=[50.,52.], bkg_uncerts=[3.,7.])
*_, CLs_obs,CLs_exp = pyhf.utils.runOnePoint(1.0, [51., 48.] + pdf.config.auxdata, pdf)
print('Observed: {} Expected: {}'.format(CLs_obs, CLs_exp[2]))
numpy_results = CLs_obs, CLs_exp[2]

Observed: [0.05290116] Expected: [0.06445521]


/home/mcf/Code/GitHub/pyhf/pyhf/tensor/numpy_backend.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(tensor_in)
/home/mcf/Code/GitHub/pyhf/pyhf/tensor/numpy_backend.py:185: RuntimeWarning: divide by zero encountered in log
  return np.exp((np.log(lam) * n) - lam - gammaln(n + 1.))


What backend is being used?

In [3]:
pyhf.get_backend()

(<pyhf.tensor.numpy_backend.numpy_backend at 0x7f6ecc835438>,
 <pyhf.optimize.opt_scipy.scipy_optimizer at 0x7f6ecc835550>)

Switch out to a different backend

In [4]:
# TensorFlow
import tensorflow as tf
sess = tf.Session()
pyhf.set_backend(pyhf.tensor.tensorflow_backend(session=sess))

In [5]:
pyhf.get_backend()

(<pyhf.tensor.tensorflow_backend.tensorflow_backend at 0x7f6e83b24ba8>,
 <pyhf.optimize.opt_tflow.tflow_optimizer at 0x7f6e83b24dd8>)

and reproduce the same result as with the NumPy backend

In [6]:
*_, CLs_obs,CLs_exp = pyhf.utils.runOnePoint(1.0, [51., 48.] + pdf.config.auxdata, pdf)
print('Observed: {} Expected: {}'.format(sess.run(CLs_obs), sess.run(CLs_exp[2])))
tensorflow_results = sess.run(CLs_obs), sess.run(CLs_exp[2])

/home/mcf/miniconda3/envs/pyhf/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Observed: [0.05257819] Expected: [0.0644623]


In [7]:
# PyTorch
pyhf.set_backend(pyhf.tensor.pytorch_backend())

In [8]:
pyhf.get_backend()

(<pyhf.tensor.pytorch_backend.pytorch_backend at 0x7f6e83b24978>,
 <pyhf.optimize.opt_pytorch.pytorch_optimizer at 0x7f6e692c0128>)

In [9]:
*_, CLs_obs,CLs_exp = pyhf.utils.runOnePoint(1.0, [51., 48.] + pdf.config.auxdata, pdf)
print('Observed: {} Expected: {}'.format(CLs_obs[0], CLs_exp[2]))
pytorch_results = CLs_obs[0], CLs_exp[2]

Observed: 0.05262800306081772 Expected: 0.06449270248413086


A comparison of the three:

In [10]:
backends = ['NumPy', 'TensorFlow', 'PyTorch']
results = [numpy_results, tensorflow_results, pytorch_results]
for backend, result in zip(backends, results):
    print('\n# {}\nObserved: {} Expected: {}'.format(backend, result[0], result[1]))


# NumPy
Observed: [0.05290116] Expected: [0.06445521]

# TensorFlow
Observed: [0.05257819] Expected: [0.0644623]

# PyTorch
Observed: 0.05262800306081772 Expected: 0.06449270248413086


# $CL_{s}$ Example using pyhf CLI

Use some preexisiting files

In [11]:
# Use some shell magics in Jupyter
% ls *.json

demo.json  new_signal.json


In [12]:
% cat demo.json

{
    "channels": [{
        "name": "singlechannel",
        "samples": [{
                "name": "sig",
                "data": [12.0, 11.0],
                "modifiers": [{
                    "name": "mu",
                    "data": null,
                    "type": "normfactor"
                }]
            },
            {
                "name": "bkg",
                "data": [50.0, 52.0],
                "modifiers": [{
                    "name": "uncorr_bkguncrt",
                    "data": [3.0, 7.0],
                    "type": "shapesys"
                }]
            }
        ]
    }],
    "data": {
        "singlechannel": [51.0, 48.0]
    },
    "toplvl": {
        "measurements": [{
            "config": {
                "poi": "mu"
            },
            "name": "singlechannel"
        }]
    }
}


In [13]:
# Use more shell magics to run from the command line
! pyhf cls demo.json

/home/mcf/Code/GitHub/pyhf/pyhf/tensor/numpy_backend.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(tensor_in)
/home/mcf/Code/GitHub/pyhf/pyhf/tensor/numpy_backend.py:185: RuntimeWarning: divide by zero encountered in log
  return np.exp((np.log(lam) * n) - lam - gammaln(n + 1.))
{
    "CLs_exp": [
        0.002606408505279359,
        0.013820656047622592,
        0.0644552079856191,
        0.23526102499555396,
        0.573041803728844
    ],
    "CLs_obs": 0.05290116065118097
}


# $CL_{s}$ with Reinterpretation

Original output

In [14]:
! pyhf cls demo.json | jq .CLs_obs

/home/mcf/Code/GitHub/pyhf/pyhf/tensor/numpy_backend.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(tensor_in)
/home/mcf/Code/GitHub/pyhf/pyhf/tensor/numpy_backend.py:185: RuntimeWarning: divide by zero encountered in log
  return np.exp((np.log(lam) * n) - lam - gammaln(n + 1.))
0.05290116065118097


Consider a new signal to test

In [15]:
% cat new_signal.json

[{
    "op": "replace",
    "path": "/channels/0/samples/0/data",
    "value": [5.0, 6.0]
}]


Apply the patch with the new signal to update the likelihood: $L \to L'$

In [16]:
! pyhf cls demo.json --patch new_signal.json | jq .CLs_obs

/home/mcf/Code/GitHub/pyhf/pyhf/tensor/numpy_backend.py:185: RuntimeWarning: divide by zero encountered in log
  return np.exp((np.log(lam) * n) - lam - gammaln(n + 1.))
/home/mcf/Code/GitHub/pyhf/pyhf/tensor/numpy_backend.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(tensor_in)
0.3401578753020146
